In [2]:
from deap import gp
import operator
import math
import numpy as np
from tqdm.notebook import trange
from time import sleep

## Primitive set

In [3]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(math.cos, 1)

pset.renameArguments(ARG0='x')

## Helpers

In [16]:
def generate_random_tree(pset, min_, max_):
    expr = gp.genHalfAndHalf(pset, min_=min_, max_=max_)
    tree = gp.PrimitiveTree(expr)
    return tree

def build_primitives_terminals_dict(pset):
    prims = dict()
    prims_funcs = list(pset.primitives.values())[0]
    prims_names = [p.name for p in prims_funcs]
    prims.update(zip(prims_names, prims_funcs))

    # for arguments, add key = value = name to the dict
    for arg in pset.arguments:
        prims[str(arg)] = arg

    return prims

def tree_to_nodes_matrix(tree: gp.PrimitiveTree, pset: gp.PrimitiveSet, prims_names: list, n_nodes=0):
    n_prims = pset.prims_count + len(pset.arguments)
    if n_nodes == 0:
        n_nodes = len(tree)
    m = np.zeros((n_nodes, n_prims))

    for i, prim in enumerate(tree):
        prim_name = prim.name.replace('ARG0', 'x')
        prim_idx = prims_names.index(prim_name)
        m[i, prim_idx] = 1.
    
    return m

def eval_fitness(tree, pset, points):
    func = gp.compile(tree, pset)

    sqerrors = ((func(x) - (x**2 + math.cos(x)))**2 for x in points)
    return math.fsum(sqerrors) / len(points)

def generate_dataset(n_samples, pset, min_, max_, points, prims_names):
    n_prims = pset.prims_count + len(pset.arguments)
    max_nodes = 2**(max_+1) - 1
    X = np.zeros((n_samples, max_nodes*n_prims))
    y = np.zeros((n_samples,1))

    trees = []
    for i in range(n_samples):
        fit = math.nan
        while math.isnan(fit) or math.isinf(fit):
            tree = generate_random_tree(pset, min_, max_)
            m = tree_to_nodes_matrix(tree, pset, prims_names, max_nodes).ravel()
            try:
                fit = eval_fitness(tree, pset, points)
            except:
                fit = math.nan

        X[i,:] = m        
        y[i,:] = fit
        trees.append(tree)

    return X, y, trees
    
    

## Generation of datasets

In [17]:
tree = generate_random_tree(pset, min_=1, max_=3)
print(tree)
prims = build_primitives_terminals_dict(pset)
tree_to_nodes_matrix(tree, pset, list(prims.keys()))

sub(cos(mul(x, x)), protectedDiv(sub(x, x), sub(x, x)))


array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [6]:
points = np.arange(0.,1.1,0.1)
print(points)
eval_fitness(tree, pset, points)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


/tmp/ipykernel_47495/2793393697.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return left / right


nan

In [7]:
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch
device = "cpu"
torch.set_default_device(device)

In [86]:
torch.manual_seed(0)
torch.set_default_dtype(torch.float64)
min_ = 1
max_ = 3
max_nodes = 2**(max_+1) - 1
n_prims = pset.prims_count + len(pset.arguments)
n_samples = 5000
X, y, trees = generate_dataset(n_samples, pset, min_, max_, points, list(prims.keys()))
y_normalized = (y - np.mean(y))/np.std(y)
frac = 0.5

generator = torch.Generator(device=device)

X_train, X_valid, X_test = random_split(X, [frac, 1-frac-0.2, 0.2], generator=generator)
y_train, y_valid, y_test = random_split(y_normalized, [frac, 1-frac-0.2, 0.2], generator=generator)


/tmp/ipykernel_47495/2793393697.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return left / right
/tmp/ipykernel_47495/2793393697.py:3: RuntimeWarning: divide by zero encountered in scalar divide
  return left / right
<string>:1: RuntimeWarning: invalid value encountered in scalar multiply


In [87]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None, target_transform=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return X[idx,:], y[idx,:]

train_dataset = CustomDataset(X_train, y_train)
valid_dataset = CustomDataset(X_valid, y_valid)
test_dataset = CustomDataset(X_test, y_test)

In [116]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(max_nodes*n_prims, 2*max_nodes*n_prims),
            nn.ReLU(),
            nn.Linear(2*max_nodes*n_prims, 2*max_nodes*n_prims),
            nn.ReLU(),
            nn.Linear(2*max_nodes*n_prims, 2*max_nodes*n_prims),
            nn.ReLU(),
            nn.Linear(2*max_nodes*n_prims, 2*max_nodes*n_prims),
            nn.ReLU(),
            nn.Linear(2*max_nodes*n_prims, max_nodes*n_prims),
            nn.ReLU(),
            nn.Linear(max_nodes*n_prims, max_nodes*n_prims//2),
            nn.ReLU(),
            nn.Linear(max_nodes*n_prims//2, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [117]:
def train_loop(dataloader, model, loss_fn, optimizer):

    ave_train_loss = 0.
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        # print(X.shape)
        # print(pred.shape)
        # print(y.shape)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ave_train_loss += loss.item()/num_batches

    return ave_train_loss

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    return test_loss

In [118]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size = None, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = None)

In [119]:
model = NeuralNetwork()
loss_fn = nn.MSELoss()

learning_rate = 1e-4
epochs = 1000

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

model.train()
pbar = trange(epochs, desc="Training", unit="epoch")
for i in pbar:
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss = test_loop(valid_dataloader, model, loss_fn)
    pbar.set_postfix(train_loss=train_loss, test_loss=test_loss)

print("Done!")

Training:   0%|          | 0/1000 [00:00<?, ?epoch/s]

KeyboardInterrupt: 

In [120]:
def test(model, test_dataloader):
    for X, y in test_dataloader:
        pred = model(X)
        print(pred.item(), y.item())

#test = np.zeros((max_nodes, n_prims))
#test[:6,:] = np.array([[1,0,0,0,0,0], [0,0,1,0,0,0], [0,0,0,0,0,1], [0,0,0,0,0,1], [0,0,0,0,1,0], [0,0,0,0,0,1]])
#test_tensor = torch.from_numpy(test.flatten())
#pred = model(test_tensor)
#pred.item()
test(model, test_dataloader)

0.9131233425776429 0.8897775097784677
0.2315477971016011 0.2512503477471599
0.7218058265677935 0.7192385748092958
0.23154953672703396 0.2303
0.7218058265677935 0.7192385748092958
0.23154953672703396 0.2303
0.7218058265677935 0.7192385748092958
0.9283635894878127 0.8527150564928959
0.7218058265677935 0.7192385748092958
0.23154953672703396 0.2303
0.7218058265677935 0.7192385748092958
0.9935221568356692 0.9864316045671913
1.4436258224563272 1.4338095849160661
0.41868800977349635 0.23208668300961277
0.2537840335195012 0.2512503477471599
0.20721413143411035 0.19833988932655025
1.4384583562040327 1.4338095849160661
0.18521933143982044 0.1885819999093278
0.2537840335195012 0.2512503477471599
0.2537840335195012 0.2512503477471599
0.30975966208905686 0.3264213578539301
2.527910825728429 2.487519097545324
0.23154953672703396 0.2303
0.2683748580848735 0.2512503477471599
0.5385010463453238 0.5303
1.4384583562040327 1.4338095849160661
0.2537840335195012 0.2512503477471599
1.4384583562040327 1.43380

In [121]:
def softmax(x):
    sm = torch.nn.Softmax(dim=1)
    with torch.no_grad():
        x_reshaped = torch.reshape(x, (max_nodes, n_prims))
    return sm(x_reshaped)

In [122]:
gamma1 = 0.
gamma2 = gamma1
def optimize_tree(x0: np.array, learning_rate, max_iter):
    x0 = torch.tensor(x0, requires_grad = True)
    optimizer_tree = torch.optim.Adam([x0], lr = learning_rate)
    #sm = torch.nn.Softmax(dim=1)
    pbar = trange(max_iter, desc="Best tree search", unit="iters")
    for i in pbar:
        pred = model(x0)
        x0_reshaped = torch.reshape(x0, (max_nodes, n_prims))
        penalty = gamma1*(torch.norm(torch.ones(x0_reshaped.shape[0])-torch.sum(x0_reshaped, dim=1)))**2 + gamma2*torch.norm(torch.maximum(x0, torch.zeros(x0.shape)))**2
        obj = pred + penalty
        obj.backward()
        print(x0.grad)
        optimizer_tree.step()
        #optimizer_tree.zero_grad()
        
        if i % 100000 == 0 and i > 0:
            with torch.no_grad():
                x_reshaped = torch.reshape(x, (max_nodes, n_prims))
                x0 = softmax(x0_reshaped).flatten().requires_grad_()
        #    print(x0)
        pbar.set_postfix(objective=pred.item())
        # print(softmax(x0))
        sleep(0.001)
    return x0

In [123]:
#tree = generate_random_tree(pset, min_=1, max_=3)
#print(tree)
x0 = tree_to_nodes_matrix(tree, pset, list(prims.keys()), n_nodes = max_nodes)
x0 = test_tensor
x = optimize_tree(x0.ravel(), 1e-4, 1)

Best tree search:   0%|          | 0/1 [00:00<?, ?iters/s]

tensor([ -6.3628e-02,   1.5401e-01,  -2.4707e-02,  -1.0302e-01,  -1.5784e-01,
        -1.4880e-316,   2.0020e-02,  -3.2839e-02,   7.9437e-02,  -7.5728e-02,
         -1.4760e-01,  -1.9514e-02,  -2.3990e-02,  -1.3183e-02,  -5.1622e-02,
          1.4798e-02,  -4.0411e-02,   6.9271e-02,   1.5822e-02,  -1.0207e-01,
         -2.1040e-02,  1.1122e-317,   7.3242e-02,   7.1728e-03,   5.5879e-02,
         -1.0471e-01,  -8.0811e-02,   1.0153e-01,  -1.0389e-01,  -8.1958e-02,
         -3.6894e-02,  -4.0406e-02,   6.1669e-02,   5.5508e-02,   3.6146e-02,
          8.3763e-02,   3.1133e-02,  -8.9631e-02,  -1.4467e-01,   4.8652e-02,
          1.3978e-02,   1.2498e-02,  -8.9659e-02,  -2.5356e-02,  -2.3306e-02,
         -2.7559e-02,   3.9765e-02,  -4.1518e-02,  -1.4318e-01,  -3.6319e-02,
         -2.7933e-02,   6.7039e-02,   2.2535e-02,   3.5434e-02,   6.9525e-02,
         -1.6789e-02,   5.5432e-02,  1.4495e-316,   1.3632e-01,  -1.6857e-01,
          1.7038e-02,   1.0221e-02,  -4.3832e-02,  1.3798e-316, 

In [76]:
#sm = torch.nn.Softmax(dim=1)
#with torch.no_grad():
#    x_reshaped = torch.reshape(x, (max_nodes, n_prims))
#    x = sm(x_reshaped)
x_reshaped = torch.reshape(x, (max_nodes, n_prims))
print(x_reshaped)
print(torch.sum(x_reshaped, dim=1))

tensor([[ 0.0987, -0.1839,  0.0568,  0.0812,  0.0820,  0.0725],
        [ 0.0561,  0.0832,  0.0913,  0.1175,  0.0640, -0.1839],
        [ 0.0717,  0.0929,  0.0911,  0.0598,  0.0882, -0.1839],
        [ 0.0926,  0.0936,  0.0921,  0.0931,  0.0932,  0.0924],
        [ 0.0909,  0.0928,  0.0934,  0.0922,  0.0908,  0.0918],
        [ 0.0906,  0.0926,  0.0928,  0.0915,  0.0912,  0.0898],
        [ 0.0908,  0.0918,  0.0920,  0.0914,  0.0914,  0.0910],
        [ 0.0892,  0.0913,  0.0891,  0.0900,  0.0912,  0.0916],
        [ 0.0896,  0.0921,  0.0928,  0.0912,  0.0914,  0.0916],
        [ 0.0905,  0.0912,  0.0904,  0.0902,  0.0894,  0.0917],
        [ 0.0934,  0.0932,  0.0915,  0.0932,  0.0932,  0.0925],
        [ 0.0918,  0.0918,  0.0933,  0.0924,  0.0905,  0.0934],
        [ 0.0935,  0.0918,  0.0927,  0.0925,  0.0915,  0.0915],
        [ 0.0921,  0.0921,  0.0921,  0.0921,  0.0921,  0.0917],
        [ 0.0915,  0.0915,  0.0915,  0.0915,  0.0915,  0.0916]],
       grad_fn=<ViewBackward0>)
tensor(

In [292]:
print(list(prims.keys()))

['add', 'sub', 'mul', 'protectedDiv', 'cos', 'x']
